<a href="https://colab.research.google.com/github/duyvm/leetcode-problems/blob/main/%5BMED%5D3394_Check_if_Grid_can_be_Cut_into_Sections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3394. Check if Grid can be Cut into Sections

https://leetcode.com/problems/check-if-grid-can-be-cut-into-sections/description/

You are given an integer `n` representing the dimensions of an `n x n` grid, with the origin at the bottom-left corner of the grid. You are also given a 2D array of coordinates `rectangles`, where `rectangles[i]` is in the form `[start`<sub>`x`</sub>`, start`<sub>`y`</sub>`, end`<sub>`x`</sub>`, end`<sub>`y`</sub>`]`, representing a rectangle on the grid. Each rectangle is defined as follows:

- `(start`<sub>`x`</sub>`, start`<sub>`y`</sub>`)`: The bottom-left corner of the rectangle.

- `(end`<sub>`x`</sub>`, end`<sub>`y`</sub>`)`: The top-right corner of the rectangle.

Note that the rectangles do not overlap. Your task is to determine if it is possible to make either **two horizontal** or **two vertical** cuts on the grid such that:

- Each of the three resulting sections formed by the cuts contains at **least one** rectangle.

- Every rectangle belongs to exactly **one section**.

Return `true` if such cuts can be made; otherwise, return `false`.

**Constraints:**
- `3 <= n <= 10`<sup>`9`</sup>
- `3 <= rectangles.length <= 10`<sup>`5`</sup>
- `0 <= rectangles[i][0] < rectangles[i][2] <= n`
- `0 <= rectangles[i][1] < rectangles[i][3] <= n`
- No two rectangles overlap.

**Example 1:**

- Input: `n = 5, rectangles = [[1,0,5,2],[0,2,2,4],[3,2,5,3],[0,4,4,5]]`

- Output: `true`

- Explanation:

    ![example1](https://drive.google.com/uc?id=1zr9yplwEPmNqIk5upOEw2akU2_p5d51G)

    The grid is shown in the diagram. We can make horizontal cuts at `y = 2` and `y = 4`. Hence, output is true.

**Example 2:**

- Input: `n = 4, rectangles = [[0,0,1,1],[2,0,3,4],[0,2,2,3],[3,0,4,3]]`

- Output: `true`

- Explanation:

    ![example2](https://drive.google.com/uc?id=11hmCL1QJ8127X6ad7u4rIC2qKxyXE3Vp)

    We can make vertical cuts at `x = 2` and `x = 3`. Hence, output is true.

**Example 3:**

- Input: `n = 4, rectangles = [[0,2,2,4],[1,0,3,2],[2,2,3,4],[3,0,4,2],[3,2,4,4]]`

- Output: `false`

- Explanation:We cannot make two horizontal or two vertical cuts that satisfy the conditions. Hence, output is false.


In [55]:
test_cases = [
    {
        "input": {
            "n": 5,
            "rectangles": [[1,0,5,2],[0,2,2,4],[3,2,5,3],[0,4,4,5]],
        },
        "output": True
    },
    {
        "input": {
            "n": 4,
            "rectangles": [[0,0,1,1],[2,0,3,4],[0,2,2,3],[3,0,4,3]],
        },
        "output": True
    },
    {
        "input": {
            "n": 4,
            "rectangles": [[0,2,2,4],[1,0,3,2],[2,2,3,4],[3,0,4,2],[3,2,4,4]],
        },
        "output": False
    },
    {
        "input": {
            "n": 6,
            "rectangles": [[4,0,5,2],[0,5,4,6],[4,5,6,6]],
        },
        "output": False
    },
    {
        "input": {
            "n": 4,
            "rectangles": [[0,0,1,1],[1,0,4,1],[0,1,2,2],[2,1,4,2],[0,2,2,4],[2,2,3,4],[3,2,4,4]],
        },
        "output": True
    },
    {
        "input": {
            "n": 5,
            "rectangles": [[0,0,1,5],[1,0,3,3],[3,0,5,3],[1,3,2,5],[2,3,5,5]],
        },
        "output": False
    },
]

# Approach 1 (Beat ~91%)

### Observations

- Given `rectangles`. Each `rectangel` has `left, right` for vertical edges and `top, bottom` for horizontal edges

- The idea is to checking edges of each rectangle in both horizontal and vertical to see if it crossed any rectangle. If not, it is a potential valid cut. Check all the edges and `count` the valid cut.

  - We only need to collect all the `right` edges and `top` edges of all `rectangles`.

  - Create two sorted `rectangles` in horizontal and vertical.

  - For each `right` edge:
  
   - Use index to tracking current edge and current rectangle in corresponding sorted `rectangles`. In this way, we only need to go through `rectangles` and `edges` once.

   - If current `edge` crosses current rectangle: we move to next `edge`

   - Else we move to next rectangle

   - If the current `edge` at the left side of new rectangle, then it is a valid cut, we increase `count` by `1`

   - If `count = 2`, return `True`

  - Do the same for horizontal edges

  - If there are no edges, return `False`

### Analysis

- Time complexity: let `m = rectangles.length`

  - One pass through `rectangles` to collect `right` and `bottom` edges: `O(m)`

  - One pass through `right` edges array for sorting: `O(log(m))` (binary sort)
  
  - One pass through `bottom` edges array for sorting: `O(log(m))` (binary sort)

  - One pass through `rectangles` for sorting based on `start`<sub>`x`</sub>: `O(log(m))` (binary sort)

  - One pass through `rectangles` for sorting based on `start`<sub>`y`</sub>: `O(log(m))` (binary sort)

  - Checking vertical cut: worst case scenario is we must check all `right` edges versus all vertically sorted `rectangles`: `O(2*m)
  
  - Checking horizonal cut: worst case scenario is we must check all `top` edges versus all horizontally sorted `rectangles`: `O(2*m)`

  - Total time complexity: `O(5*m + 4*log(m)) ~ O(m + log(m))`

### Implementation

In [67]:
from typing import List, Optional
from collections import defaultdict
from collections import deque
from heapq import heapify, heappush, heappop
from queue import PriorityQueue

class Solution:
    def checkValidCuts(self, n: int, rectangles: List[List[int]]) -> bool:
        vertical_edges = set()
        horizontal_edges = set()

        for start_x, start_y, end_x, end_y in rectangles:
            vertical_edges.add(end_x)
            horizontal_edges.add(end_y)

        vertical_edges = list(vertical_edges)
        vertical_edges.sort()
        horizontal_edges = list(horizontal_edges)
        horizontal_edges.sort()

        rectangles_sort_vertically = sorted(rectangles, key=lambda x: x[0])
        rectangles_sort_horizontally = sorted(rectangles, key=lambda x: x[1])

        # check vertical cut
        count = 0
        edge_idx = 0
        rect_idx = 0
        while edge_idx < len(vertical_edges)-1 and rect_idx < len(rectangles):
            vertical_edge = vertical_edges[edge_idx]
            rect = rectangles_sort_vertically[rect_idx]

            if rect[0] >= vertical_edge:
                count += 1
                edge_idx += 1
            elif rect[0] < vertical_edge < rect[2]:
                edge_idx += 1
            else:
                rect_idx += 1

            if count == 2:
                return True

        # check horizontal cut
        count = 0
        edge_idx = 0
        rect_idx = 0
        while edge_idx < len(horizontal_edges)-1 and rect_idx < len(rectangles):
            horizontal_edge = horizontal_edges[edge_idx]
            rect = rectangles_sort_horizontally[rect_idx]

            if rect[1] >= horizontal_edge:
                count += 1
                edge_idx += 1
            elif rect[1] < horizontal_edge < rect[3]:
                edge_idx += 1
            else:
                rect_idx += 1

            if count == 2:
                return True

        return False

In [ ]:
solution = Solution()
for i, test_case in enumerate(test_cases):
    result = solution.checkValidCuts(test_case["input"]["n"], test_case["input"]["rectangles"])
    if result != test_case["output"]:
        print(f'Failed test case {i} with input: {test_case["input"]} and expected result: {test_case["output"]} vs actual result: {result}')

In [ ]:
solution = Solution()
tc = 2
result = solution.checkValidCuts(test_cases[tc]["input"]["n"], test_cases[tc]["input"]["rectangles"])
if result != test_case["output"]:
    print(f'Failed test case {i} with input: {test_cases[tc]["input"]} and expected result: {test_cases[tc]["output"]} vs actual result: {result}')